In [1]:
import sys

sys.path.append("../scripts")
from utils import group_dict, get_meta, build_label, save_label

from glob import glob
import pandas as pd
import json
import os
from os import makedirs
from os.path import join


# Building json Dataset from meta.json files

In [2]:
ROOT = "/home/timssh/ML/TAGGING/data_valid"
SOURCE = "/home/timssh/ML/TAGGING/source"
META = "meta"
# list of picsets
picsets = glob(ROOT + f"/*/*/{META}.json")
picsets.extend(glob(ROOT + f"/*/*/*/{META}.json"))
len(picsets), picsets[-2:], group_dict


(89,
 ['/home/timssh/ML/TAGGING/data_valid/body/skinny/skinny-girl-anal-porn-/meta.json',
  '/home/timssh/ML/TAGGING/data_valid/body/fat/very-fat-gilrs-solo-porn-nude/meta.json'],
 {'body_type': ['normal body',
   'fat',
   'curvy',
   'skinny',
   'bodybuilder',
   'athletic'],
  'hair_color': ['brown hair',
   'blue hair',
   'red hair',
   'violet hair',
   'green hair',
   'blonde hair',
   'black hair',
   'pink hair'],
  'hair_type': ['wavy hair', 'curly haired', 'straight hair'],
  'sex_positions': ['missionary',
   'cowgirl',
   'spooning',
   'doggy style',
   'reverse cowgirl',
   '69'],
  'tits_size': ['huge tits', 'small tits', 'flat chested', 'big tits']})

In [3]:
def label_builder(group, item, picset):
    path = '/'.join(picset.split('/')[:-1]) + '/picture/' + item['Id'] 
    if os.path.exists(path + '.jpeg'):
        path += '.jpeg'
    elif os.path.exists(path + '.jpg'):
        path += '.jpg'
    if path.split('.')[-1] not in ['jpg', 'jpeg']:
        return None
    
    series_dict = {
        'path': path
    }
    
    for key in group_dict[group['group']]:
        if key in group['category']:
            series_dict[key] = 1
        else:
            series_dict[key] = 0
    return series_dict

def save_meta(meta_dict):    
    for key in meta_dict.keys():
        df = pd.DataFrame(meta_dict[key])
        num2label, weights = build_label(df)
        makedirs(join(SOURCE, key), exist_ok=True)
        save_label(df.to_json(), df.to_json(), num2label, weights, key, SOURCE)
        

In [4]:
df_dict = {}
for group in group_dict.keys():
    if group != "group_of_girls":
        df_dict[group] = []

for picset in picsets:
    meta = get_meta(picset)

    for item in meta["items"]:
        flag = 0
        for train_ in item['trained']:
            if train_['group'] == 'group_of_girls' and train_['category'][0] == "one girl":
                flag = 1
        if flag == 1:
            for group in group_dict.keys():
                tr = 0
                for train in item["trained"]:
                    # if group['group'] == 'group_of_girls' and group['category'][0] == "one girl":

                    if group == train["group"]:
                        l_b = label_builder(train, item, picset)
                        if l_b is not None:
                            df_dict[train["group"]].append(l_b)
                        tr = 1
                if tr == 0:
                    l_b = label_builder({"group": group, "category": []}, item, picset)
                    if l_b is not None:
                        df_dict[group].append(l_b)


In [5]:
GROUP = 'hair_color'
df_dict[GROUP]


[{'path': '/home/timssh/ML/TAGGING/data_valid/hair-colour/violet/pornoaktrisa-s-fioletovymi-volosami-porn/picture/646da259c6052b7308f99019f7ec76a9.jpeg',
  'brown hair': 0,
  'blue hair': 0,
  'red hair': 0,
  'violet hair': 1,
  'green hair': 0,
  'blonde hair': 0,
  'black hair': 0,
  'pink hair': 0},
 {'path': '/home/timssh/ML/TAGGING/data_valid/hair-colour/violet/pornoaktrisa-s-fioletovymi-volosami-porn/picture/7afd109fa81489e83e2114c5cdceaa00.jpeg',
  'brown hair': 0,
  'blue hair': 0,
  'red hair': 0,
  'violet hair': 1,
  'green hair': 0,
  'blonde hair': 0,
  'black hair': 0,
  'pink hair': 0},
 {'path': '/home/timssh/ML/TAGGING/data_valid/hair-colour/violet/pornoaktrisa-s-fioletovymi-volosami-porn/picture/50a54b3fa322f53574fcd8d10e431f88.jpeg',
  'brown hair': 0,
  'blue hair': 0,
  'red hair': 0,
  'violet hair': 0,
  'green hair': 0,
  'blonde hair': 0,
  'black hair': 0,
  'pink hair': 1},
 {'path': '/home/timssh/ML/TAGGING/data_valid/hair-colour/violet/pornoaktrisa-s-fiole

In [6]:
len(df_dict[GROUP])


12149

In [7]:
df_dict.keys()


dict_keys(['body_type', 'hair_color', 'hair_type', 'sex_positions', 'tits_size'])

In [8]:
# save_meta(df_dict)


In [9]:
df = pd.DataFrame(df_dict[GROUP])
df


,path,brown hair,blue hair,red hair,violet hair,green hair,blonde hair,black hair,pink hair
0,/home/timssh/ML/TAGGING/data_valid/hair-colour...,0,0,0,1,0,0,0,0
1,/home/timssh/ML/TAGGING/data_valid/hair-colour...,0,0,0,1,0,0,0,0
2,/home/timssh/ML/TAGGING/data_valid/hair-colour...,0,0,0,0,0,0,0,1
3,/home/timssh/ML/TAGGING/data_valid/hair-colour...,0,0,0,1,0,0,0,0
4,/home/timssh/ML/TAGGING/data_valid/hair-colour...,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
12144,/home/timssh/ML/TAGGING/data_valid/body/fat/ve...,0,0,0,0,0,0,1,0
12145,/home/timssh/ML/TAGGING/data_valid/body/fat/ve...,0,0,0,0,0,1,0,0
12146,/home/timssh/ML/TAGGING/data_valid/body/fat/ve...,1,0,0,0,0,0,0,0
12147,/home/timssh/ML/TAGGING/data_valid/body/fat/ve...,0,0,0,0,0,0,1,0


In [10]:
import json

with open(f"/home/timssh/ML/TAGGING/source_old/{GROUP}/{GROUP}.json") as json_file:
    json_ = json.load(json_file)

data = pd.read_json(json_["train"])
print(json_["num2label"], json_["weights"])
data


{'0': 'black-hair', '1': 'blonde-hair', '2': 'blue-hair', '3': 'brown-hair', '4': 'green-hair', '5': 'pink-hair', '6': 'red-hair', '7': 'violet-hair'} [3.696663740122915, 4.554353704705246, 9.547619047619047, 9.103783783783785, 22.883152173913043, 10.539424280350438, 7.754143646408839, 35.98717948717949]


,dir,hair_black,hair_blonde,hair_blue,hair_brown,hair_green,hair_pink,hair_redhead,hair_violet
0,/home/timssh/ML/TAGGING/data/noise/mens-tits/p...,0,0,0,0,0,0,0,0
1,/home/timssh/ML/TAGGING/data/hair/brown/light-...,0,0,0,1,0,0,0,0
2,/home/timssh/ML/TAGGING/data/hair/black/brunet...,1,0,0,0,0,0,0,0
3,/home/timssh/ML/TAGGING/data/hair/black/brunet...,1,0,0,0,0,0,0,0
4,/home/timssh/ML/TAGGING/data/hair/green/green/...,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
10144,/home/timssh/ML/TAGGING/data/hair/blue/blue-ha...,0,0,1,0,0,0,0,0
10145,/home/timssh/ML/TAGGING/data/hair/green/green-...,0,0,0,0,1,0,0,0
10146,/home/timssh/ML/TAGGING/data/hair/blonde/blond...,0,1,0,0,0,0,0,0
10147,/home/timssh/ML/TAGGING/data/hair/blue/blue/pi...,0,0,1,0,0,0,0,0


In [14]:
col_dict = {
    "dir": "path",
    "hair_black": "black hair",
    "hair_blonde": "blonde hair",
    "hair_blue": "blue hair",
    "hair_brown": "brown hair",
    "hair_green": "green hair",
    "hair_pink": "pink hair",
    "hair_redhead": "red hair",
    "hair_violet": "violet hair",
}
# col_dict = {
#     "dir": "path",
#     "boobs_big-tits": "big tits",
#     "boobs_flat-chested": "flat chested",
#     "boobs_huge-tits": "huge tits",
#     "boobs_small-tits": "small tits",
# }
# col_dict = {
#     "dir": "path",
#     "hair-type_curly-hair": "curly haired",
#     "hair-type_straight-hair": "straight hair",
#     "hair-type_wawy-hair": "wavy hair",
# }
# col_dict = {
#     "dir": "path",
#     "poses_69": "69",
#     "poses_cowgirl": "cowgirl",
#     "poses_doggy-style": "doggy style",
#     "poses_missionary" : "missionary",
#     "poses_reverse-cowgirl" : "reverse cowgirl", 
#     "poses_spooning" : "spooning",
# }
# col_dict = {
#     "dir": "path",
#     "body_athletic-body": "athletic",
#     "body_bodybilding-body": "bodybuilder",
#     "body_curvy": "curvy",
#     "body_fat" : "fat",
#     "body_normal-body" : "normal body", 
#     "body_skinny" : "skinny",
# }

data.columns = [col_dict[key] for key in data.columns.to_list()]
data = data[df.columns.to_list()]
data

,path,brown hair,blue hair,red hair,violet hair,green hair,blonde hair,black hair,pink hair
0,/home/timssh/ML/TAGGING/data/noise/mens-tits/p...,0,0,0,0,0,0,0,0
1,/home/timssh/ML/TAGGING/data/hair/brown/light-...,1,0,0,0,0,0,0,0
2,/home/timssh/ML/TAGGING/data/hair/black/brunet...,0,0,0,0,0,0,1,0
3,/home/timssh/ML/TAGGING/data/hair/black/brunet...,0,0,0,0,0,0,1,0
4,/home/timssh/ML/TAGGING/data/hair/green/green/...,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
10144,/home/timssh/ML/TAGGING/data/hair/blue/blue-ha...,0,1,0,0,0,0,0,0
10145,/home/timssh/ML/TAGGING/data/hair/green/green-...,0,0,0,0,1,0,0,0
10146,/home/timssh/ML/TAGGING/data/hair/blonde/blond...,0,0,0,0,0,1,0,0
10147,/home/timssh/ML/TAGGING/data/hair/blue/blue/pi...,0,1,0,0,0,0,0,0


In [15]:
new_tits = pd.concat([data, df])
new_tits.reset_index(inplace=True, drop=True)
new_tits


,path,brown hair,blue hair,red hair,violet hair,green hair,blonde hair,black hair,pink hair
0,/home/timssh/ML/TAGGING/data/noise/mens-tits/p...,0,0,0,0,0,0,0,0
1,/home/timssh/ML/TAGGING/data/hair/brown/light-...,1,0,0,0,0,0,0,0
2,/home/timssh/ML/TAGGING/data/hair/black/brunet...,0,0,0,0,0,0,1,0
3,/home/timssh/ML/TAGGING/data/hair/black/brunet...,0,0,0,0,0,0,1,0
4,/home/timssh/ML/TAGGING/data/hair/green/green/...,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
22293,/home/timssh/ML/TAGGING/data_valid/body/fat/ve...,0,0,0,0,0,0,1,0
22294,/home/timssh/ML/TAGGING/data_valid/body/fat/ve...,0,0,0,0,0,1,0,0
22295,/home/timssh/ML/TAGGING/data_valid/body/fat/ve...,1,0,0,0,0,0,0,0
22296,/home/timssh/ML/TAGGING/data_valid/body/fat/ve...,0,0,0,0,0,0,1,0


In [16]:
num2label, weights = build_label(new_tits)
new_source = '/home/timssh/ML/TAGGING/source_valid'
makedirs(join(new_source, GROUP), exist_ok=True)
save_label(new_tits.to_json(), new_tits.to_json(), num2label, weights, GROUP, new_source)

hair_color


In [17]:
weights

[5.440108401084011,
 12.51496259351621,
 8.254111842105264,
 67.1371237458194,
 31.863492063492064,
 4.222549432057215,
 3.589771101573677,
 18.70829450139795]

In [18]:
num2label

{0: 'brown hair',
 1: 'blue hair',
 2: 'red hair',
 3: 'violet hair',
 4: 'green hair',
 5: 'blonde hair',
 6: 'black hair',
 7: 'pink hair'}